# Leikecase

Datasett: *Marine Mammals Sightings in and around Svalbard 1995-2019*
Inneheld rapporterte observasjoner av marine pattedyr på og rundt Svalbard i tidsperioden 1995-2019. Dataen består av tidspunkt, lokasjon, art og antall individ observert.

Kolonne | Forklaring
-- |-- 
eventID | Unik identifikator
coordinateUncertaintyInMeters | Usikkerheit i koordinatane
decimalLatitude | Breddegrad i WGS-84. -90.0 til 90.0
decimalLongitude | Lengegrad i WGS-84. -180.0 til 180.0
scientificName | Artsnamn på latinsk
eventDate | Tidspunkt for observasjonen.
individualCount | Antall individ observert

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# Tving pandas til å vise alle kolonnene i output
pd.set_option('display.max_columns', None)

## Hente data

Mål:
- Laste inn CSV for observasjonar
- Laste inn CSV for artsnamn på norsk

In [ ]:
observations = pd.read_csv("data/MMSDB_observations_1995-2019-v2.csv", delimiter="$")
animals = pd.read_csv("data/animals.csv")

Oppgåver:
- Kor mange observasjonar har det blitt gjort totalt?
- Kor mange ulike arter har det blitt rapportert observasjonar på?

In [ ]:
observations.info()

In [ ]:
observations["scientificName"].unique()

In [ ]:
animals.head()

## Rense og forberede data

Mål:
- Fjerne alle rader utan koordinatar eller observasjonstidspunkt
- Parse tidspunkt til pandas.Timestamp
- Joine inn norske namn til observasjonane
- Opprett ein dataframe som kun ser på isbjørnar

In [ ]:
observations = observations[
    (~observations["eventDate"].isna())
    & (~observations["decimalLongitude"].isna())
    & (~observations["decimalLatitude"].isna())
]

In [ ]:
observations = pd.merge(observations, animals, on="scientificName")

In [ ]:
observations.columns

In [ ]:
observations.describe()

In [ ]:
observations["timestamp"] = observations["eventDate"].apply(lambda ed: pd.Timestamp(ed))

In [ ]:
observations.head()

## Transformere data

- Opprett ei ny kolonne "year" basert på tidspunkt
- Lag ein ny dataframe som er aggregert per art og år.
- Per gruppe:
    - Summer opp totalt antall individ
    - Tell opp kor mange observasjonar som har blitt gjort

In [ ]:
observations["year"] = observations["timestamp"].apply(lambda t: t.year)
observations = observations[
    (observations["year"] >= 1995) &
    (observations["year"] <= 2019)
]
observations_isbjørn = observations[observations["norwegianName"] == "Isbjørn"]

In [ ]:
observations_isbjørn_grouped = observations_isbjørn.groupby(["norwegianName", "year"]).agg(
    total_individuals=("individualCount", np.sum),
    num_sightings=("eventID", np.count_nonzero)
).reset_index().sort_values(by=["norwegianName", "year"], ascending=True)

In [ ]:
observations_isbjørn_grouped

In [ ]:
observations_piv = observations.pivot_table(
    "individualCount", 
    index="year", 
    columns="norwegianName", 
    aggfunc="sum"
)

In [ ]:
observations_piv.fillna(0, inplace=True)

In [ ]:
observations_piv.head()

## Plotting og visualisering

- Plott antall observerte isbjørnar per år
- Per år, plott antall isbjørnar og hvalrossar
- Etter 2017, vis alle obervasjonar for isbjørnar i folium
    - Rekn ut middels breddegrad og lengdegrad
    - Plott alle observasjonar
    - Legg til tidspunkt, koordinatar og antall individ i popup
    - Lagre kart som til disk

### Antall isbjørnar per år

In [ ]:
observations_isbjørn_grouped.plot(x="year", secondary_y=["individuals_per_sighting"])

### Isbjørnar og kvalrossar

In [ ]:
observations_subplots = observations_piv[["Isbjørn", "Hvalross"]]

In [ ]:
observations_subplots.plot()

### Folium og kart

In [ ]:
mean_latitude = observations_isbjørn["decimalLatitude"].mean()
mean_latitude

In [ ]:
mean_longitude = observations_isbjørn["decimalLongitude"].mean()
mean_longitude

In [ ]:
import folium

m = folium.Map(
    location=(mean_latitude, mean_longitude),
    zoom_start=4.5
)
m

In [ ]:
for idx, row in observations_isbjørn[
    observations_isbjørn["year"] >= 2017
].iterrows():
    lat = row["decimalLatitude"]
    lng = row["decimalLongitude"]
    folium.Marker(
        location=[lat, lng],
        popup=f"""
        Timestamp: {row["timestamp"]}
        Individuals: {row["individualCount"]}
        Location: ({lat:.3f}, {lng:.3f})
        """
    ).add_to(m)

In [ ]:
m

In [ ]:
m.save("data/map.html")